In [1]:
import torch
from dataloader import get_dataloader
from configurate import get_config
import torchvision.transforms as trs
from easydict import EasyDict

In [2]:
args = EasyDict({
    'iterations' : 100,
    'classes_per_it_tr' : 60,
    'num_support_tr' : 5,
    'num_query_tr' : 5,
    'classes_per_it_val' : 5,
    'num_support_val' : 5,
    'num_query_val' : 15,
})

In [3]:
train_loader, val_loader=  get_dataloader(args,'train','val')


Making dataset: 100%|██████████| 688/688 [00:05<00:00, 133.31it/s]


train
val
